### Notebook for initializing polymer systems using a DPD potential

In [1]:
import matplotlib
import numpy as np  
import scipy.stats
import gsd, gsd.hoomd 
import freud 
import math
import itertools 
import hoomd 
import fresnel 
%matplotlib inline
matplotlib.style.use("ggplot")
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats("svg")

In [2]:
import time

In [3]:
def render(frame): 
    '''
    Visualize a simulation box within a jupyter notebook.
    
    '''
    scene = fresnel.Scene()
    geometry = fresnel.geometry.Sphere(scene, N=frame.particles.N)
    geometry.material = fresnel.material.Material(color=fresnel.color.linear([0.01, 0.74, 0.26]), roughness=0.5)
    geometry.position[:] = frame.particles.position
    geometry.outline_width = 0.01
    box = fresnel.geometry.Box(scene, frame.configuration.box,box_radius=0.01)
    L = frame.configuration.box[0]
    scene.camera = fresnel.camera.Perspective(position=(L*1.8, L*1.8, L * 2.2), look_at=(0, 0, 0), up=(0, 1, 0), height=0.28)

    if frame.bonds.N>0:
        geometry.radius[:] = [0.2]*frame.particles.N 

        all_bonds = np.stack(
        [
            frame.particles.position[frame.bonds.group[:, 0]],
            frame.particles.position[frame.bonds.group[:, 1]],
        ],
        axis=1,
        )
        bond_distances = np.linalg.norm(all_bonds[:,0,:]-all_bonds[:,1,:], axis=1)
        L = frame.configuration.box[0]
        bond_indices = np.where(bond_distances < L/2)[0]
        filtered_bonds = all_bonds[bond_indices, :, :]
        
        bonds = fresnel.geometry.Cylinder(scene, N=len(filtered_bonds))
        bonds.material = fresnel.material.Material(roughness=0.5)
        bonds.outline_width = 0.05

        bonds.points[:] = filtered_bonds
        bonds.radius[:] = [0.1]*len(filtered_bonds)
        bonds.material.primitive_color_mix = 1.0
        bonds.color[:] = fresnel.color.linear([0.8, 0.8, 0.8])
                
    return fresnel.preview(scene)

In [4]:
def initialize_snapshot(num_pol, num_mon, density=0.85):
    
    Create a HOOMD snapshot of a cubic box with the number density given by input parameters.

    
    N = num_pol * num_mon
    L = np.cbrt(N / density)  # Calculate box size based on density
    buffer = 4
    print(L)
    #TODO: change to random walk
    positions = np.random.uniform(low=-(0.5), high=(0.5), size=(N, 3))
    bonds = []
    for i in range(num_pol):
        start = i * num_mon
        for j in range(num_mon - 1):
            bonds.append([start + j, start + j + 1])
    bonds = np.array(bonds)
    frame = gsd.hoomd.Frame()
    frame.particles.types = ['A']
    frame.particles.N = N
    frame.particles.position = positions
    frame.bonds.N = len(bonds)
    frame.bonds.group = bonds
    frame.bonds.types = ['b']
    frame.configuration.box = [L, L, L, 0, 0, 0]
    return frame

IndentationError: unexpected indent (3372220738.py, line 1)

In [14]:
import mbuild as mb
from mbuild.path import HardSphereRandomWalk
from mbuild.utils.geometry import bounding_box
from mbuild.utils.volumes import (
    CuboidConstraint)

def rand_walk_inside_cube(num_pol,num_mon,density):
    N = num_pol * num_mon
    L = np.cbrt(N / density)  # Calculate box size based on density
    buffer = 0.5
    cube = CuboidConstraint(Lx=L-0.5, Ly=L-0.5, Lz=L-0.5)
    n_chains = num_pol
    chain_length = num_mon
    seeds = [np.random.randint(low=1) for i in range(n_chains)]
    chains = []
    for seed in seeds:    
        rw_path = HardSphereRandomWalk(
                N=chain_length,
                bond_length=0.9,
                radius=0.1,
                volume_constraint=cube,
                min_angle=np.pi / 4,
                max_angle=np.pi,
                max_attempts=1e3,
                seed=seed,
            )
        rw_path.generate()
        
        chain = Polymer()
        chain.build_from_path(path=rw_path, energy_minimize=True)
        chains.append(chain)
        
    bounds = bounding_box(rw_path.coordinates)
    return chains, bounds

In [15]:
chains = rand_walk_inside_cube(num_pol=20,num_mon=10,density=0.8)

IndexError: index 10 is out of bounds for axis 0 with size 10

In [12]:
def pbc(d,box):
    for i in range(3):
        a = d[:,i]
        a[a < -box[i]/2] += box[i]
        a[a >  box[i]/2] -= box[i]
    return d

In [18]:
def check_bond_length_equillibration(gsdfile,bond_length,num_mon,num_pol):
    fene_bond_l = bond_length
    traj = gsd.hoomd.open(gsdfile)
    last_frame = traj[-1]
    frame_ds = []
    for j in range(num_pol):
        idx = j*num_mon
        d1 = last_frame.particles.position[idx:idx+num_mon-1] - last_frame.particles.position[idx+1:idx+num_mon]
        bond_l = np.linalg.norm(pbc(d1,frame.configuration.box),axis=1)
        frame_ds.append(bond_l)
    max_frame_bond_l = np.max(np.array(frame_ds))
    min_frame_bond_l = np.min(np.array(frame_ds))
    print("max: ",max_frame_bond_l," min: ",min_frame_bond_l)
    if max_frame_bond_l <= fene_bond_l and min_frame_bond_l >= (fene_bond_l - 0.3):
        print("DPD Simulation Finished.")
        return True
    if max_frame_bond_l > fene_bond_l or min_frame_bond_l < (fene_bond_l - 0.3):
        return False

## Parameters

In [33]:
num_pol = 50
num_mon = 50
kT = 2.0
cut = 2.5
l = 1.0 #bond length r_0
timestep = 0.001
gamma = 0.1 #DPD parameter
A = 50.0 #DPD parameter

## HOOMD simulation code

In [15]:
frame = initialize_snapshot(num_pol, num_mon, density=density)

Box size: 26.099


In [16]:
print(frame.particles.position)

[[[ 4.25173437  5.21912758  6.29612458]]

 [[ 3.64647929  4.42609712  6.22706685]]

 [[ 4.36163872  4.7688364   5.61790614]]

 ...

 [[ 9.17980774  3.64908694 -4.17329195]]

 [[ 9.92287521  3.25663167 -3.63123144]]

 [[10.62613407  3.96705578 -3.65814912]]]


In [19]:
start_time = time.perf_counter

num_pol = 400
num_mon = 40
kT = 1.0
cut = 1.2
l = 1.0 #bond length r_0
timestep = 0.001
gamma = 400.0 #DPD parameter
A = 5000.0 #DPD parameter, A scales down with N
k=10000
density = 0.9

frame = initialize_snapshot(num_pol, num_mon, density=density)

#assign harmonic bond potentials
harmonic = hoomd.md.bond.Harmonic()
harmonic.params["b"] = dict(r0=l, k=k)

integrator = hoomd.md.Integrator(dt=timestep)
integrator.forces.append(harmonic)

#initialize HOOMD simulation object
simulation = hoomd.Simulation(device=hoomd.device.auto_select(),seed=12)
simulation.operations.integrator = integrator 
simulation.create_state_from_snapshot(frame) #create initial state of Simulation

#add ensemble method to Simulation
const_vol = hoomd.md.methods.ConstantVolume(filter=hoomd.filter.All())
integrator.methods.append(const_vol)

#define the neighbor list
nlist = hoomd.md.nlist.Cell(buffer=0.4)
simulation.operations.nlist = nlist

#define the forcefield
DPD = hoomd.md.pair.DPD(nlist, default_r_cut=cut, kT=kT)
DPD.params[('A', 'A')] = dict(A=A, gamma=gamma)
integrator.forces.append(DPD)
simulation.operations.integrator = integrator

#define write frequency for trajectory file
gsd_out = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(10), 
    mode='wb',
    dynamic=['property','momentum'],
    filename='run_len_%s_pol_%s.gsd'%(num_mon,num_pol),
    truncate=False)
simulation.operations.writers.append(gsd_out)

#run the simulation! and log by flushing the writers
simulation.run(100)
gsd_out.flush()

fene_bond_l = 1.2
file = 'run_len_%s_pol_%s.gsd'%(num_mon,num_pol)
count = 1000


while not check_bond_length_equillibration(file,fene_bond_l,num_mon,num_pol):
    simulation.run(1000)
    gsd_out.flush()
    count += 1000
render(simulation.state.get_snapshot())
print(f"Finished in {count} iterations.")
end_time = time.perf_counter
total_time = end_time-start_time
print("total time: ",total_time)

Box size: 26.099
max:  1.5931346  min:  0.44762433
max:  1.1212424  min:  0.8070919
max:  1.0691397  min:  0.8710645
max:  1.0636678  min:  0.909832
DPD Simulation Finished.
Finished in 4000 iterations.


## Run code

In [37]:
#run the simulation! and log by flushing the writers
simulation.run(1_000)
gsd_out.flush()

fene_bond_l = 1.2
file = 'run_len_%s_pol_%s.gsd'%(num_mon,num_pol)
count = 1000


while not check_bond_length_equillibration(file,fene_bond_l,num_mon,num_pol):
    simulation.run(200)
    gsd_out.flush()
    count += 200
render(simulation.state.get_snapshot())
print(f"Finished in {count} iterations.")

max bond length:  7.683396
min bond length:  0.23592383
max bond length:  7.369175
min bond length:  0.21894705
max bond length:  7.587952
min bond length:  0.27974385
max bond length:  7.50972
min bond length:  0.2194792
max bond length:  8.6888075
min bond length:  0.16744503
max bond length:  7.4796405
min bond length:  0.14268649
max bond length:  7.717604
min bond length:  0.15104882
max bond length:  7.7832747
min bond length:  0.21767738
max bond length:  7.772977
min bond length:  0.2623106
max bond length:  7.37987
min bond length:  0.16633241
max bond length:  7.5375447
min bond length:  0.17321211
max bond length:  7.41851
min bond length:  0.14127648
max bond length:  7.1190305
min bond length:  0.3377385
max bond length:  6.418019
min bond length:  0.19830942
max bond length:  7.3047056
min bond length:  0.2899017
max bond length:  6.6087494
min bond length:  0.20736666
max bond length:  6.550147
min bond length:  0.28899005
max bond length:  6.0029664
min bond length:  0.

KeyboardInterrupt: 

### Graph the bond length relaxation

In [ ]:
import matplotlib.pyplot as plt

with gsd.hoomd.open('run_len_%s_pol_%s.gsd'%(num_mon,num_pol),'r') as traj:
    avg_frame_bond_l = []
    for frame in traj:
        bond_l = []
        for i in range(0,num_mon-2):
            dr = np.linalg.norm((frame.particles.position[i+1] - frame.particles.position[i]), axis=0)
            bond_l.append(dr)
        avg_frame_bond_l.append(np.mean(bond_l))

plt.plot(avg_frame_bond_l)